Property consists of attributes; location Street address --> (co-ordinates), price, area.
these attriutes need to be revalued to the heat map intensity scales, this can be done with maximum land value and minimum land value per SQM. 
[longatude, latitude, intensity]

Market consists of attributes; average land price, average land area, value per SQM. 

Property attributes and market attributes define characteristics of the valuation map. 

Converting street address into co-ordinates is done with the Bing Maps api. 

The csv file has the property data: location and price, the location is plugged into the the address tab and the price into the price catagory. 
The intensity should be auto calculated, maybe set to the standard deviation of the data set. 

A web crawler is set up to pull sale data off of p24. 

In [9]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm 

# Define the number of iterations
num_iterations = 1

pricelist = []
locationlist = []
addresslist = []

# Create a tqdm progress bar
for x in tqdm(range(num_iterations), desc="Scraping Progress"):
    p24 = f"https://www.property24.com/houses-for-sale/gauteng/1/p{x}"
    req = requests.get(p24)
    soup = BeautifulSoup(req.text, "html.parser")

    for location in soup.find_all("span", attrs=["p24_location"]):
        locationlist.append(location.text)

    for price in soup.find_all("span", attrs=["p24_price"]):
        pricelist.append(price.text)
    
    for location in soup.find_all("div", class_="p24_location"):
        locationlist.append(location.text)

    for location in soup.find_all("div", class_="p24_price"):
        locationlist.append(location.text)
        

pricelist = [price.strip().replace("\n", "").replace("\r", "").replace(" ", "").replace("R", "").replace("POA", "0") for price in pricelist]    

csvdata = [[addr, price] for addr, price in set(zip(pricelist, locationlist))]

print(f"CSVDATA: {len(csvdata), csvdata}")

Scraping Progress: 100%|██████████| 1/1 [00:01<00:00,  1.34s/it]

CSVDATA: (4, [['2499000', 'Mabopane  Unit X'], ['860000', 'Midlands Estate'], ['1910275', 'Magaliesberg Country Estate'], ['1190000', 'Franschhoek']])


Using the propattr class the median and sd are calculated for the data. 

The data should be sorted by area name to avoid over use of the api. 

The data is sorted by name and pulled into its own list to find the co-ordinates using Bings Map Api.

In [10]:
import geocoder
from tqdm import tqdm

commonlocations = []


for data in csvdata:
    if data[1] not in commonlocations:
        commonlocations.append(data[1])

print(f"SIZE: {len(commonlocations)} {commonlocations}")

common_coords = []

failed = []

for addr in tqdm(commonlocations, desc="Finding Coordinates"):
    try:
        g = geocoder.bing(f'{addr}', key="Au_zOIgH37lhBFI2w8tNQxjZRxFwuD7892Hx7mBZV6ifLba99oxRQFh6uryjZ5cD")
        
        # Check if the geocoding request was successful
        if g.ok:
            results = g.json
            common_coords.append([addr, results])
        else:
            # Handle the case where the geocoding request failed
            print(f"Geocoding request failed for address: {addr}")
            failed.append(addr)
    except Exception as e:
        # Handle any exceptions that may occur during the geocoding request
        print(f"An error occurred while geocoding address: {addr}")
        print(f"Error details: {str(e)}")

SIZE: 4 ['Mabopane  Unit X', 'Midlands Estate', 'Magaliesberg Country Estate', 'Franschhoek']


Finding Coordinates: 100%|██████████| 4/4 [00:02<00:00,  1.78it/s]


In [11]:
import numpy as np

class marketattr:
    def __init__(self, market, averagesqm, maxsqm, minsqm, props=[]):
        self.market = market or None
        self.averagesqm = averagesqm or None
        self.maxsqm = maxsqm or None
        self.minsqm = minsqm or None
        self.props = props
        
    def addprop(self, prop):
        self.props.append(prop)


class propattr(marketattr):
    def __init__(self, coords, price, address) -> None:
        self.address = address
        self.coords = coords or self.get_coords(self.address)
        self.price = price
        self.intensity = self.calculate_intensity()

    def calculate_intensity(self):
        medianlist = np.array(pricelist, dtype=int)
        median = np.median(medianlist)
        stdprice = np.std(medianlist)
        
        if int(self.price) < int(median):
            return 1
        
        if int(self.price) > int(median) + int(stdprice):
            return 2
        
        if int(self.price) > int(median) + 2 * int(stdprice):
            return 3
        
        if int(self.price) > int(median) + 3 * int(stdprice):
            return 4
        
        else:
            return 1
    
    def get_coords(self, addr):
        for address in common_coords:
            if addr == address[0]:
                coords = [address[1]["lat"], address[1]["lng"]]
                return coords
        

market = marketattr("Gauteng", None, None, None, props=[])

In [12]:
with open(f"prop_{market.market}_locations_price.csv",  "w") as r:
    fieldnames = ["Location", "Price"]
    writer = csv.writer(r)
    for row in csvdata:
        writer.writerow(row)
       
df = []

with open(f"prop_{market.market}_locations_price.csv", "r") as r:
    reader = csv.reader(r)
    
    for data in reader:
        if len(data) > 1:
            df.append(data)

print(pricelist)
print(df)

['1190000', '1910275', '2499000', '860000']
[['2499000', 'Mabopane  Unit X'], ['860000', 'Midlands Estate'], ['1910275', 'Magaliesberg Country Estate'], ['1190000', 'Franschhoek']]


In [13]:
with open("data_attached.csv", "w") as w:
    writer = csv.writer(w)

    for row in common_coords:
        writer.writerow(row)

In [14]:
comcoords = []
with open("data_attached.csv", "r") as w:
    writer = csv.reader(w)

    for row in common_coords:
        comcoords.append(row)

The market is built with the data collected.

In [15]:
def buildprops():
    prop = propattr(coords=None, price=props[0], address=f"{props[1]}")
    market.addprop(prop)
    print(prop.coords)
    

for props in tqdm(df, desc="Building props"):
    buildprops()
print("Done.")
    

Building props: 100%|██████████| 4/4 [00:00<00:00, 999.83it/s]

[-25.5087986, 28.06579208]
[-25.93249893, 28.18990135]
[-25.6695652, 28.07815361]
[-33.91412354, 19.12263489]
Done.


Using the map showcased earlier, we are able to input the three value printed above to start forming the heat map. 

In [16]:
from ipyleaflet import Map, basemaps, Heatmap
center = [-25.97529793, 28.11886787]
zoom = 13
heatmap_data = []
location_data = []

m = Map(basemap=basemaps.Esri.WorldStreetMap, center=center, zoom=zoom)

for prop in market.props:
    try:
        if prop is not None:
            print(prop.coords[0], prop.coords[1], prop.intensity)
            sprop = prop.coords[0], prop.coords[1], prop.intensity
            heatmap_data.append(sprop)
    except TypeError as e:
               pass

m.add_layer(Heatmap(locations=heatmap_data, radius=100, blur=80))

m

-25.5087986 28.06579208 2
-25.93249893 28.18990135 1
-25.6695652 28.07815361 1
-33.91412354 19.12263489 1


Map(center=[-25.97529793, 28.11886787], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_ti…